## An introduction to traits

- A powerful library
- Open source
- [Enthought](https://www.enthought.com)
- Part of ETS: Enthought Tool Suite


## ETS: Enthought Tool Suite

- https://docs.enthought.com/ets/
- Traits: Object Models
- TraitsUI: Views for Objects having Traits
- Chaco: 2D Visualizations
- Mayavi: 3D Visualizations

<br/>

- Envisage: Application Framework
- Miscellaneous libraries


## Introduction to Traits

- **trait**: Python object attribute with additional characteristics

<br/>

- https://docs.enthought.com/traits/
- https://github.com/enthought/traits/


## Trait features

- Initialization: default value
- Validation: strongly typed
- Deferral/Delegation: value delegation
- Notification: events
- Visualization: MVC, automatic GUI!


## An example


In [ ]:
from traits.api import (Delegate, HasTraits,
    Instance, Int, Str, observe)

class Parent(HasTraits):
    # INITIALIZATION: 'last_name' initialized to ''
    last_name = Str('')


In [ ]:
class Child(HasTraits):
    age = Int
    # VALIDATION: 'father' must be Parent instance
    father = Instance(Parent)
    # DELEGATION: 'last_name' delegated to father's
    last_name = Delegate('father')
    # NOTIFICATION: Method called when 'age' changes
    def _age_changed(self, old, new):
        print('Age changed from %s to %s ' % (old, new))


## Using this


In [ ]:
joe = Parent()
joe.last_name = 'Johnson'
moe = Child()
moe.father = joe

In [ ]:
# Delegation
moe.last_name

In [ ]:
# Notification
moe.age = 10

In [ ]:
# Validation
moe.age = '1'

In [ ]:
# Visualization
moe.configure_traits()

- Live editing!

In [ ]:
%gui qt

In [ ]:
moe.edit_traits()

In [ ]:
moe.age = 21

## Predefined trait types

- Standard: `Bool, Complex, Int, Float, Str, Tuple, List, Dict`
- Constrained: `Range, Regex, Expression, ReadOnly`
- Special: `Date, Either/Union, Enum, Array, File, Color, Font, Button`
- Generic: `Instance, Any, Callable`
- Custom traits: 2D/3D plots etc.


## Trait change notification

- Static: `def _<trait_name>_changed()`
- Decorator: `@observe('extended.trait.name')`
- Dynamic:

```obj.observe(handler, 'extended.trait.name')
```

- See documentation: https://docs.enthought.com/traits/traits_user_manual/notification.html


## Notification example


In [ ]:
class Parent(HasTraits):
    last_name = Str('')


class Child(HasTraits):
    age = Int
    father = Instance(Parent)

    def _age_changed(self, old, new):
        print('Age changed from %s to %s ' % (old, new))

    @observe('father.last_name')
    def _dad_name_updated(self, event):
        print('DAD name', self.father.last_name)


In [ ]:
def handler(event):
    print("handler", event.object, event.name, event.old, event.new)

In [ ]:
c.age = 21
c.father = Parent(last_name='Shyam')

In [ ]:
c = Child(father=Parent(last_name='Ram'))
c.observe(handler, 'father, age')

## Exercise

- Modify the first example to produce the above example
- Add a `first_name` trait
- Add a `Bool` trait to specify if person is alive
- Add an `Enum` for the gender of the child



## Solution


In [ ]:
from traits.api import Bool, Enum

class Parent(HasTraits):
    last_name = Str('')


class Child(HasTraits):
    age = Int
    father = Instance(Parent)
    first_name = Str('')
    alive = Bool(True)
    gender = Enum('female', 'male', 'neither')

    def _age_changed(self, old, new):
        print('Age changed from %s to %s ' % (old, new))

    @observe('father.last_name')
    def _dad_name_updated(self, event):
        print('DAD name', self.father.last_name)


In [ ]:
p = Parent(last_name='Ray')
c = Child(age=21, father=p, first_name='Romano', gender='male')

## Setting default values

- For simple cases, use the default of the trait
- For more complex cases use a special method
- A simple example


In [ ]:
import datetime

from traits.api import HasTraits, Date, Range

class Thing(HasTraits):
    date = Date()
    age = Int(12)

    def _date_default(self):
        print('default')
        return datetime.datetime.today()

In [ ]:
t = Thing()

In [ ]:
type(c.age)

## Trait Lists



In [ ]:
from traits.api import List

class Bowl(HasTraits):
    fruits = List(Str)

    def _fruits_changed(self, o, n):
        print("Fruits changed", o, n)


In [ ]:
b = Bowl()
b.fruits = ['apple']
b.fruits.append('mango')

## Trait List events


In [ ]:
class Bowl(HasTraits):
    fruits = List(Str)
    def _fruits_changed(self, o, n):
        print("Fruits changed", o, n)

    def _fruits_items_changed(self, list_event):
        print(list_event.index)
        print(list_event.removed)
        print(list_event.added)


In [ ]:
b = Bowl()
b.fruits = ['apple']
b.fruits.append('mango')

In [ ]:
def handler(event):
    print("h:", event)

b.observe(handler, 'fruits.items')
b.fruits.append('peach')

In [ ]:
# Remove the handler
b.observe(handler, 'fruits.items', remove=True)

## Other events

- `TraitChangeEvent`
- `ListChangeEvent`
- `DictChangeEvent`
- `SetChangeEvent`


## Property traits

- What if you have a quantity that is computed?
- Use Property traits here
- Use the `observe=` kwarg
- Use `@cached_property` to cache output
- Use the `_get_propname` and `_set_propname`


In [ ]:
from math import pi
from traits.api import Range, Float, Property, cached_property

class Circle(HasTraits):
    radius = Range(0.0, 1000.0)
    area = Property(Float, observe='radius')

    @cached_property
    def _get_area(self):
        print("computing area")
        return pi*self.radius**2

In [ ]:
c = Circle(radius=2)
c.area

In [ ]:
c.area

## Array traits

- Can handle numpy arrays of arbitrary shape
- Example of the Lissajous curves
- Consider, $x = sin (at + \delta)$, $y=sin(b t)$


In [ ]:
import numpy as np
from traits.api import Array, Range, observe

class Lissajous(HasTraits):
    tmax = Range(1.0, 1000.0, value=10.0)
    n = Range(10, 1000)
    x = Array(dtype=float, shape=(None,))
    y = Array(dtype=float, shape=(None,))
    a = Float(1.0)
    b = Float(1.0)
    delta = Float(0.0)

    @observe('tmax, n, a, b, delta')
    def update(self, event=None):
        t = np.linspace(0, self.tmax, self.n)
        self.x = np.sin(self.a*t + self.delta)
        self.y = np.sin(self.b*t)

In [ ]:
lj = Lissajous()
lj.a = 0.5

In [ ]:
lj.edit_traits()

## So what?

- Let us now make a plot


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from traits.api import Any, Array, Range, observe

class Lissajous(HasTraits):
    tmax = Range(1.0, 1000.0, value=10.0)
    n = Range(10, 1000)
    x = Array(dtype=float, shape=(None,))
    y = Array(dtype=float, shape=(None,))
    a = Float(1.0)
    b = Float(1.0)
    delta = Float(0.0)
    plot = Any

    @observe('tmax, n, a, b, delta')
    def update(self, event=None):
        t = np.linspace(0, self.tmax, self.n)
        self.x = np.sin(self.a*t + self.delta)
        self.y = np.sin(self.b*t)
        if self.plot is None:
            self.plot, = plt.plot(self.x, self.y)
        else:
            self.plot.set_data(self.x, self.y)
            fig = plt.gcf()
            fig.canvas.draw()


In [ ]:
import matplotlib
%matplotlib qt

In [ ]:
l = Lissajous()

In [ ]:
l.edit_traits()

In [ ]:
plt.clf()
plot, = plt.plot(l.x, l.y)

In [ ]:
plot.set_data(l.x, l.y)

In [ ]:
l.a = 0.5
l.n = 100

## Full solution


In [ ]:
from matplotlib import pyplot as plt
from traits.api import Any, Array, HasTraits, Range, Float

class Lissajous(HasTraits):
    tmax = Range(1.0, 1000.0, value=10.0)
    n = Range(10, 10000, value=100)
    x = Array(dtype=float, shape=(None,))
    y = Array(dtype=float, shape=(None,))
    a = Float(1.0)
    b = Float(1.0)
    delta = Float(0.0)
    plot = Any(None)

    @observe('tmax, n, a, b, delta')
    def update(self, event):
        t = np.linspace(0, self.tmax, self.n)
        self.x = np.sin(self.a*t + self.delta)
        self.y = np.sin(self.b*t)
        if self.plot is None:
            self.plot, = plt.plot(self.x, self.y)
        else:
            self.plot.set_data(self.x, self.y)
            fig = plt.gcf()
            fig.canvas.draw()

In [ ]:
lj = Lissajous()
lj.edit_traits()

## More information

- Only scratched the surface
- Tutorial: https://docs.enthought.com/traits/traits_tutorial/index.html
- User manual: https://docs.enthought.com/traits/traits_user_manual
- https://github.com/enthought/traits
- Clone git repo; run `examples/tutorials/tutor.py`
- And many examples with a UI


## The traits demos

<img src="images/traits_demos.png" height="90%"/>


## Exercise/homework

- Take your julia set code (or the Mandelbrot set)
- Make these into parameters
  - The domain size (xmin, xmax, ymin, ymax)
  - The maximum iterations
  - The size of the array
  - The constant value used, $C$ (for the Julia set)
- Run the computation when any of these change